In [2]:
import os
import re
import pandas as pd

In [3]:
folder = f"{os.path.expanduser('~')}/nlu/project/texts/snd/"

In [4]:
file_tsv = os.path.join(folder,
                   '0179MalikIbnAnas.Muwatta.IbnZayd.ShamAY0033922_manual-SND.tsv')

In [5]:
file_conll = os.path.join(folder,
                   '0179MalikIbnAnas.Muwatta.IbnZayd.ShamAY0033922_manual-SND.conll')

In [109]:
manual_snd = f"{os.path.expanduser('~')}/nlu/project/texts/snd/manual_snd/"

In [51]:
for file in os.listdir(folder):
    if file.endswith('.tsv'):
        
        token_label = []
        
        with open(os.path.join(folder, file), 'r') as f:
            tsv = f.read()
            
            for idx, lines in enumerate(tsv.split('\n')):
                
                if not lines.startswith('#') and len(lines) > 0:
                    span_t, span_chr, token, label, _ = tuple(lines.split('\t'))
                    
                    token_label.append(token + ' ' + label[0] + '\n')
            
             with open(os.path.join(folder, file[:-4] + '.conll'), 'w') as out:
                    out.writelines(token_label)
            
            

In [111]:
def df_to_conll(df, filename, directory):
    df.to_csv(os.path.join(directory, filename),
              sep='\t', header=False, index=False)

In [88]:
df_to_conll(df, 'TRIAL.conll', folder)x

In [113]:
def clean_conll(path_to_file):
    
    df = pd.read_csv(path_to_file, sep=' ', header=None)
    df.columns = ['tokens', 'labels']
    
    for idx, row in df.iterrows():
        if df.at[idx, 'labels'] == 'S':
            if df.at[idx-1, 'labels'] == '_':
                df.at[idx, 'labels'] = 'B'
            elif df.at[idx+1, 'labels'] == '_':
                 df.at[idx, 'labels'] = 'E'
            else:
                df.at[idx, 'labels'] = 'I'
                
    arabicRegex = "[ذ١٢٣٤٥٦٧٨٩٠ّـضصثقفغعهخحجدًٌَُلإإشسيبلاتنمكطٍِلأأـئءؤرلاىةوزظْلآآ]+|#"
    
    df.loc[df.tokens.str.contains(arabicRegex, regex = True)]
    
    df.loc[df.tokens.str.match('#'), 'labels'] = '#'
    
    for idx, row in df.iterrows():
        if df.at[idx, 'labels'] == '#' and df.at[idx+1, 'labels'] == '#':
            df.drop(idx, inplace=True)
            
    df = df.replace(['#'], ' ')
    
    return df
    

In [114]:
for file in os.listdir(folder):
    if file.endswith('.conll'):
        dataframe = clean_conll(os.path.join(folder, file))
        
        df_to_conll(dataframe, file, manual_snd)